<a href="https://colab.research.google.com/github/pascal-maker/SAM-Med3D/blob/main/openmedlab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! git clone https://github.com/openmedlab/MedLSAM

Cloning into 'MedLSAM'...
remote: Enumerating objects: 405, done.
remote: Counting objects: 100% (220/220), done.
remote: Compressing objects: 100% (138/138), done.
remote: Total 405 (delta 117), reused 170 (delta 78), pack-reused 185
Receiving objects: 100% (405/405), 5.10 MiB | 17.06 MiB/s, done.
Resolving deltas: 100% (209/209), done.


In [2]:
# Install PyTorch
!pip install torch

# Clone the MedLSAM repository
!git clone https://github.com/openmedlab/MedLSAM

# Enter the MedLSAM folder
%cd MedLSAM

# Install dependencies
!pip install -e .


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [3]:
!pip install torch>=1.11.0
!pip install tqdm
!pip install nibabel
!pip install scipy
!pip install SimpleITK
!pip install monai


In [5]:
# Install gdown to download files from Google Drive
!pip install gdown

# Download the first file from Google Drive
!gdown --id 1cwoHS5yNPI22-jgMnoUo_yqj0U9Rab3E

# Download the second file from the direct link
!wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth

# Download the folder from Google Drive
!gdown --folder https://drive.google.com/drive/folders/1ETWmi4AiniJeWOt6HAsYgTjYv_fkgzoN


/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1cwoHS5yNPI22-jgMnoUo_yqj0U9Rab3E
From (redirected): https://drive.google.com/uc?id=1cwoHS5yNPI22-jgMnoUo_yqj0U9Rab3E&confirm=t&uuid=477d45d5-da03-4043-9b6b-0a3d092ad455
To: /content/MedLSAM/medlam.pth
100% 60.3M/60.3M [00:02<00:00, 30.1MB/s]
--2024-06-02 17:01:03--  https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.35.18.102, 13.35.18.15, 13.35.18.103, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.35.18.102|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 375042383 (358M) [binary/octet-stream]
Saving to: ‘sam_vit_b_01ec64.pth.1’

sam_vit_b_01ec64.pt 100%[===================>] 3

In [6]:
# Create separate directories for each checkpoint
!mkdir -p /content/MedLSAM/checkpoint/medlam
!mkdir -p /content/MedLSAM/checkpoint/sam_vit_b_01ec64
!mkdir -p /content/MedLSAM/checkpoint/medsam_vit_b

# Move medlam.pth to its checkpoint directory
!mv /content/MedLSAM/medlam.pth /content/MedLSAM/checkpoint/medlam/medlam.pth

# Move sam_vit_b_01ec64.pth to its checkpoint directory
!mv /content/MedLSAM/sam_vit_b_01ec64.pth /content/MedLSAM/checkpoint/sam_vit_b_01ec64/sam_vit_b_01ec64.pth

# Move medsam_vit_b.pth to its checkpoint directory
!mv /content/MedLSAM/MedSAM/medsam_vit_b.pth /content/MedLSAM/checkpoint/medsam_vit_b/medsam_vit_b.pth


In [7]:
from google.colab import drive

drive.mount('/content/drive')


Mounted at /content/drive


In [8]:
# Unzip the dataset (assuming it's already uploaded to /content/Task1_HaN_OAR.zip)
!unzip "/content/drive/MyDrive/Colab Notebooks/Task1_HaN_OAR.zip" -d "/content/dataset"

# Prepare the Data
import os
from glob import glob

# Define paths
dataset_path = '/content/dataset'
support_image_path = '/content/MedLSAM/config/data/StructSeg_HaN/support_image.txt'
support_label_path = '/content/MedLSAM/config/data/StructSeg_HaN/support_label.txt'
query_image_path = '/content/MedLSAM/config/data/StructSeg_HaN/query_image.txt'
query_label_path = '/content/MedLSAM/config/data/StructSeg_HaN/query_label.txt'

# Create the directories if they don't exist
os.makedirs(os.path.dirname(support_image_path), exist_ok=True)

# List all image and label files
image_files = sorted(glob(os.path.join(dataset_path, 'image_*.nii.gz')))
label_files = sorted(glob(os.path.join(dataset_path, 'label_*.nii.gz')))

# Split into support and query groups (e.g., first 3 as support, rest as query)
support_images = image_files[:3]  # You can change this to select a different number of support images
support_labels = label_files[:3]
query_images = image_files[3:]
query_labels = label_files[3:]

# Write the file paths to text files
with open(support_image_path, 'w') as f:
  for item in support_images:
    f.write("%s\n" % item)

with open(support_label_path, 'w') as f:
  for item in support_labels:
    f.write("%s\n" % item)

with open(query_image_path, 'w') as f:
  for item in query_images:
    f.write("%s\n" % item)

with open(query_label_path, 'w') as f:
  for item in query_labels:
    f.write("%s\n" % item)

# Create the Configuration File
config_path = '/content/MedLSAM/config/data/StructSeg_HaN/config.txt'

config_content = """
[data]
support_image_ls  =  config/data/StructSeg_HaN/support_image.txt
support_label_ls  =  config/data/StructSeg_HaN/support_label.txt
query_image_ls    =  config/data/StructSeg_HaN/query_image.txt
query_label_ls    =  config/data/StructSeg_HaN/query_label.txt
gt_slice_threshold = 10
bbox_mode             = SPL
slice_interval        = 2
fg_class              = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22]
seg_save_path         = result/npz/StructSeg
seg_png_save_path     = result/png/StructSeg

[vit]
net_type                = vit_b

[weight]
medlam_load_path   = /content/MedLSAM/checkpoint/medlam/medlam.pth
vit_load_path  = /content/MedLSAM/checkpoint/sam_vit_b_01ec64/sam_vit_b_01ec64.pth
"""

# Write the config content to a file
os.makedirs(os.path.dirname(config_path), exist_ok=True)
with open(config_path, 'w') as f:
  f.write(config_content)


Archive:  /content/drive/MyDrive/Colab Notebooks/Task1_HaN_OAR.zip
   creating: /content/dataset/HaN_OAR/
   creating: /content/dataset/HaN_OAR/1/
 extracting: /content/dataset/HaN_OAR/1/data.nii.gz  
  inflating: /content/dataset/HaN_OAR/1/label.nii.gz  
   creating: /content/dataset/HaN_OAR/10/
 extracting: /content/dataset/HaN_OAR/10/data.nii.gz  
  inflating: /content/dataset/HaN_OAR/10/label.nii.gz  
   creating: /content/dataset/HaN_OAR/11/
 extracting: /content/dataset/HaN_OAR/11/data.nii.gz  
  inflating: /content/dataset/HaN_OAR/11/label.nii.gz  
   creating: /content/dataset/HaN_OAR/12/
 extracting: /content/dataset/HaN_OAR/12/data.nii.gz  
  inflating: /content/dataset/HaN_OAR/12/label.nii.gz  
   creating: /content/dataset/HaN_OAR/13/
 extracting: /content/dataset/HaN_OAR/13/data.nii.gz  
  inflating: /content/dataset/HaN_OAR/13/label.nii.gz  
   creating: /content/dataset/HaN_OAR/14/
 extracting: /content/dataset/HaN_OAR/14/data.nii.gz  
  inflating: /content/dataset/HaN_O

In [10]:
import os

# Define paths
config_structseg_path = '/content/MedLSAM/config/test_config/test_structseg_medlam.txt'
config_word_path = '/content/MedLSAM/config/test_config/test_word_medlam.txt'

# Configuration content for StructSeg MedLAM
config_structseg_content = """
[data]
support_image_ls = /content/MedLSAM/config/data/StructSeg_HaN/support_image.txt
support_label_ls = /content/MedLSAM/config/data/StructSeg_HaN/support_label.txt
query_image_ls = /content/MedLSAM/config/data/StructSeg_HaN/query_image.txt
query_label_ls = /content/MedLSAM/config/data/StructSeg_HaN/query_label.txt
gt_slice_threshold = 10
bbox_mode = SPL
slice_interval = 2
fg_class = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22]
seg_save_path = result/npz/StructSeg
seg_png_save_path = result/png/StructSeg

[vit]
net_type = vit_b

[weight]
medlam_load_path = /content/MedLSAM/checkpoint/medlam/medlam.pth
vit_load_path = /content/MedLSAM/checkpoint/sam_vit_b_01ec64/sam_vit_b_01ec64.pth
"""

# Configuration content for WORD MedLAM
config_word_content = """
[data]
support_image_ls = /content/MedLSAM/config/data/WORD/support_image.txt
support_label_ls = /content/MedLSAM/config/data/WORD/support_label.txt
query_image_ls = /content/MedLSAM/config/data/WORD/query_image.txt
query_label_ls = /content/MedLSAM/config/data/WORD/query_label.txt
gt_slice_threshold = 100
fg_class = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]
bbox_mode = WPL
slice_interval = False
feature_scale_ls = ['feature0', 'feature1', 'feature2']
seg_save_path = result/npz/WORD
seg_png_save_path = result/png/WORD

[vit]
net_type = vit_b

[weight]
medlam_load_path = /content/MedLSAM/checkpoint/medlam/medlam.pth
vit_load_path = /content/MedLSAM/checkpoint/sam_vit_b_01ec64/sam_vit_b_01ec64.pth
"""

# Create directories if they don't exist
os.makedirs(os.path.dirname(config_structseg_path), exist_ok=True)
os.makedirs(os.path.dirname(config_word_path), exist_ok=True)

# Write the updated config content to the files
with open(config_structseg_path, 'w') as f:
    f.write(config_structseg_content.strip())

with open(config_word_path, 'w') as f:
    f.write(config_word_content.strip())

# Change directory to MedLSAM
os.chdir('/content/MedLSAM')

# Run the inference commands
os.system('CUDA_VISIBLE_DEVICES=0 python MedLAM_Inference.py --config_file /content/MedLSAM/config/test_config/test_structseg_medlam.txt')
os.system('CUDA_VISIBLE_DEVICES=0 python MedLAM_Inference.py --config_file /content/MedLSAM/config/test_config/test_word_medlam.txt')


256

In [15]:


# Run the inference commands
!CUDA_VISIBLE_DEVICES=0 python MedLAM_Inference.py --config_file /content/MedLSAM/config/test_config/test_structseg_medlam.txt


data support_image_ls /content/MedLSAM/config/data/StructSeg_HaN/support_image.txt /content/MedLSAM/config/data/StructSeg_HaN/support_image.txt
data support_label_ls /content/MedLSAM/config/data/StructSeg_HaN/support_label.txt /content/MedLSAM/config/data/StructSeg_HaN/support_label.txt
data query_image_ls /content/MedLSAM/config/data/StructSeg_HaN/query_image.txt /content/MedLSAM/config/data/StructSeg_HaN/query_image.txt
data query_label_ls /content/MedLSAM/config/data/StructSeg_HaN/query_label.txt /content/MedLSAM/config/data/StructSeg_HaN/query_label.txt
data gt_slice_threshold 10 10
data bbox_mode SPL SPL
data slice_interval 2 2
data fg_class [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22] [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22]
data seg_save_path result/npz/StructSeg result/npz/StructSeg
data seg_png_save_path result/png/StructSeg result/png/StructSeg
vit net_type vit_b vit_b
weight medlam_load_path /content/MedLSAM/checkpoint/med

In [16]:
!CUDA_VISIBLE_DEVICES=0 python /content/MedLSAM/MedLSAM_WPL_Inference.py --config_file /content/MedLSAM/config/test_config/test_structseg_medlam_wpl_medsam.txt
!CUDA_VISIBLE_DEVICES=0 python /content/MedLSAM/MedLSAM_WPL_Inference.py --config_file /content/MedLSAM/config/test_config/test_structseg_medlam_wpl_medsam.txt
!CUDA_VISIBLE_DEVICES=0 python /content/MedLSAM/MedLSAM_SPL_Inference.py --config_file /content/MedLSAM/config/test_config/test_structseg_medlam_spl_sam.txt
!CUDA_VISIBLE_DEVICES=0 python /content/MedLSAM/MedLSAM_SPL_Inference.py --config_file /content/MedLSAM/config/test_config/test_structseg_medlam_spl_sam.txt


data support_image_ls config/data/StructSeg_HaN/support_image.txt config/data/StructSeg_HaN/support_image.txt
data support_label_ls config/data/StructSeg_HaN/support_label.txt config/data/StructSeg_HaN/support_label.txt
data query_image_ls config/data/StructSeg_HaN/query_image.txt config/data/StructSeg_HaN/query_image.txt
data query_label_ls config/data/StructSeg_HaN/query_label.txt config/data/StructSeg_HaN/query_label.txt
data gt_slice_threshold 10 10
data fg_class [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22] [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22]
data bbox_mode WPL WPL
data slice_interval False False
data seg_save_path result/npz/StructSeg/medlam_medsam result/npz/StructSeg/medlam_medsam
data seg_png_save_path result/png/StructSeg/medlam_medsam result/png/StructSeg/medlam_medsam
vit net_type vit_b vit_b
weight medlam_load_path checkpoint/medlam.pth checkpoint/medlam.pth
weight vit_load_path checkpoint/medsam_vit_b.pth checkpoint